Introduction
============

C'est une triste histoire bien connue: tous les cent ans, le village de
Thierceville est envahi de lycanthropes se transformant en bêtes
féroces. Toute personne mordue par une telle créature les rejoint
aussitôt, rejoignant les rangs des créatures des ténèbres.

Mais cette fois-ci, les villageois ont décidé d'anticiper, et ont invité
des apothicaires à s'installer dans le village. La réputation du village
a aussi attiré un certain nombre de chasseurs de monstres qui comptent
bien débarrasser les lieux de tout danger.

Dans ce TP, nous vous proposons de concevoir une simulation du village
de Thierceville comprenant les villageois, les loups-garous, les
chasseurs et les apothicaires. La simulation du village sera très
simplifiée: l'environnement ne sera pas représenté. L'intéraction entre
agents se limitera à des actions extrêmement simples et les agents ne
chercheront psa à se coordonner. Ces différents aspects des systèmes
multi-agents feront l'objet du second TP.

Afin de modéliser le village de Thierceville, nous utiliserons la
plate-forme `mesa`. `mesa` est une plate-forme multi-agents utilisée
pour modéliser notamment des agents situés. Elle intègre notamment un
environnement graphique facile d'utilisation ainsi que la possibilité de
tracer des courbes sur la simulation. `mesa` est écrite en python et
inclut un client léger pour voir les graphiques et la simulation.


Mise en place
=============

Pour ce TP et les suivants, vous aurez besoin d'une version récente de
python 3, plus spécifiquement une version de python supérieure ou égale
à la version 3.7. Si vous n'avez pas encore de version de python
installée, je vous recommande [Anaconda](https://www.anaconda.com/). Vous pouvez également
installer un IDE si vous le souhaitez, ou développer directement depuis
un éditeur de texte en lançant les scripts depuis la ligne de commande.

Une fois python installé, vous devrez installer le package `mesa`. Pour
ce faire, ouvrez un terminal et tapez:

```bash
pip install -r requirements.txt
```

En cas de besoin ou de doute, vous trouverez la documentation de `mesa`
ici: <https://mesa.readthedocs.io/en/master/>.

Une fois GAMA lancé, téléchargez importez le TP1 situé sur le git
data-ensta:

```bash
git clone foo
```

Les librairies à importer sont les suivantes:

In [2]:
import math
import random
from collections import defaultdict

import tornado, tornado.ioloop
import mesa
import numpy
import pandas
import uuid
from mesa import space
from mesa.batchrunner import BatchRunner
from mesa.datacollection import DataCollector
from mesa.time import RandomActivation
from mesa.visualization.ModularVisualization import ModularServer, VisualizationElement
from mesa.visualization.modules import ChartModule

Mesa contient la description de plusieurs éléments:

- **Le modèle** qui représente les éléments communs à la simulation, qu'il s'agisse
    des paramètres en termes de nombres d'agents ou des éléments qui en
    sont indépendants. Le modèle est aussi doté d'un *scheduler* qui est
    chargé d'activer les agents. Dans le cadre de ce cours, nous
    utiliserons systématiquement le `RandomActivation` scheduler qui
    active les agents dans un ordre aléatoire.


In [5]:
class Village(mesa.Model):
    def __init__(self, n_villagers):
        mesa.Model.__init__(self)
        self.space = mesa.space.ContinuousSpace(600, 600, False)
        self.schedule = RandomActivation(self)
        for _ in range(n_villagers):
            self.schedule.add(Villager(random.random() * 500, random.random() * 500, 10, int(uuid.uuid1()), self))

    def step(self):
        self.schedule.step()
        if self.schedule.steps >= 1000:
            self.running = False


- **Les agents** sont des classes qui héritent de la classe `mesa.Agent`. Les autres
    éléments incluent surtout des éléments de visualisation: dans ce
    modèle comme dans les suivants vous verrez notamment le canvas que
    nous utiliserons, c'est à dire l'espace dans lequel se déplacent les
    agents. Cet élément correspond à un élément présent dans le
    JavaScript, de la même manière que les éléments comme les cercles et
    rectangles utilisés pour représenter les agents.

In [4]:
def wander(x, y, speed, model):
    r = random.random() * math.pi * 2
    new_x = max(min(x + math.cos(r) * speed, model.space.x_max), model.space.x_min)
    new_y = max(min(y + math.sin(r) * speed, model.space.y_max), model.space.y_min)

    return new_x, new_y


class Villager(mesa.Agent):
    def __init__(self, x, y, speed, unique_id: int, model: Village):
        super().__init__(unique_id, model)
        self.pos = (x, y)
        self.speed = speed
        self.model = model

    def portrayal_method(self):
        color = "blue"
        r = 3
        portrayal = {"Shape": "circle",
                     "Filled": "true",
                     "Layer": 1,
                     "Color": color,
                     "r": r}
        return portrayal

    def step(self):
        self.pos = wander(self.pos[0], self.pos[1], self.speed, self.model)

Vous verrez plusieurs éléments pour chaque agent:

-   la méthode `step` indique ce que l'agent fait à son tour
-   la méthode `portrayal method` indique la manière dont l'agent doit
    être représenté sur le simulateur.

- Le **cannevas** représente la représentation de l'environnement faite sur l'interface graphique. Elle permet aussi de définir où se trouve le javascript dont nous avons besoin pour notre modèle. **Vous n'aurez pas à modifier cette classe**:

In [5]:
class ContinuousCanvas(VisualizationElement):
    local_includes = [
        "./js/simple_continuous_canvas.js",
    ]

    def __init__(self, canvas_height=500,
                 canvas_width=500, instantiate=True):
        VisualizationElement.__init__(self)
        self.canvas_height = canvas_height
        self.canvas_width = canvas_width
        self.identifier = "space-canvas"
        if (instantiate):
            new_element = ("new Simple_Continuous_Module({}, {},'{}')".
                           format(self.canvas_width, self.canvas_height, self.identifier))
            self.js_code = "elements.push(" + new_element + ");"

    def portrayal_method(self, obj):
        return obj.portrayal_method()

    def render(self, model):
        representation = defaultdict(list)
        for obj in model.schedule.agents:
            portrayal = self.portrayal_method(obj)
            if portrayal:
                portrayal["x"] = ((obj.pos[0] - model.space.x_min) /
                                  (model.space.x_max - model.space.x_min))
                portrayal["y"] = ((obj.pos[1] - model.space.y_min) /
                                  (model.space.y_max - model.space.y_min))
            representation[portrayal["Layer"]].append(portrayal)
        return representation

Le bloc `main` sert quant à lui de définir la manière dont les blocs
graphiques s'agencent. Pour le moment, seul un unique bloc graphique est
présent (celui du simulateur). Nous en ajouterons dans la suite de ce
TP. Le serverur sert aussi à régler les valeurs des options du modèle,
ici le nombre de villageois. Nous verrons dans la suite du TP la manière
de régler ces éléments directement dans l'interface graphique du client. Le lancement du serveur est fait dans une fonction afin de pouvoir lancer les batchs dans la suite du TP (*cf.* ci-dessous)



In [6]:

def run_single_server():
    server = ModularServer(Village,
                           [ContinuousCanvas()],
                           "Village",
                           {"n_villagers": 25})
    server.port = 8521
    server.launch()

    
if __name__ == "__main__":
    run_single_server()


Interface starting at http://127.0.0.1:8521


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}


Cela lancera d'un côté le serveur python qui fera tourner le code du
modèle et des agents, et le code du client html/JavaScript. Cela devrait
aussi lancer votre navigateur sur la page du simulateur. Si la page ne
s"ouvre pas et que le serveur python s'est lancé, ouvrez votre
navigateur et ouvrez l'URL `http://127.0.0.1:8521/`.

L'interface graphique de mesa se présente comme la figure ci-dessous. L'entête de la
page continent le nom du modèle (ici Village). Le bouton `About` permet
d'avoir accès à la description du modèle. Cela peut en particulier
permettre d'expliquer à un utilisateur tiers ce qui est vu à l'écran et
l'analyser. Sur la partie droite de l'entête, les trois boutons
permettent de contrôler le déroulement de la simulation: un bouton
permettant de lancer ou mettre en pause la sumilation, un permettant de
faire un unique tour et un permettant de la remettre à zéro.

Sous l'entête se trouve une barre permettant de régler le nombre de
tours de simulation par seconde. Plus ce nombre sera important, plus la
simulation ira vite. Notez cependant qu'un nombre de tours par seconde
réglé à 0 permettra de faire tourner la simulation à sa vitesse
maximale. Le nombre de tours écoulés se trouve sous la barre. Au-dessous
se trouve la fenêtre de simulation, où on peut voir les agents

![./GUI.png](./GUI.png)

Il vous est possible de stopper le serveur avec les lignes ci-dessous. Notez que jupyter vous indiquera que le kernel est stoppé et qu'il sera relancé. Acceptez.

In [7]:
import tornado, tornado.ioloop
tornado.ioloop.IOLoop.current().stop()

Implémentation de la simulation
===============================

La simulation donnée est très simple, et ne donne une représentation que
des agents `Villagers`. La première partie de ce TP consiste à la
compléter.

Les lycanthropes
----------------

La première tâche consistera à implémenter le fait qu'une personne
puisse être un lycanthrope. Pour cela, nous allons enrichir l'espèce
personne. Ajoutez à la classe `Villager` un booléen indiquant
si la personne est un loup-garou. Faites en sorte que les lycanthropes
soient affichés en rouge. Il doit y avoir 5 lycanthropes au départ de la
simulation. Dans le modèle, il vous est possible d'ajouter un paramètre
de la même manière que `n_villagers`. Ce paramètre doit figurer
dans le constructeur du modèle `Village.__init__`, mais aussi
dans la création de ce modèle, sous la forme d'une chaîne de caractère
dans la liste constituant le dernier paramètre du constructeur de
`ModularServer`.

``` python
class Village(mesa.Model):
    def __init__(self, n_villagers):
        mesa.Model.__init__(self)
        #...
        for _ in range(n_villagers):
            self.schedule.add(Villager(random.random() * 600,
                random.random() * 600, 10,
                random.randint(1, 600), self))
#...
if __name__ == "__main__":
        server = ModularServer(Village,
                           [ContinuousCanvas],
                           "Village",
                           {"n_villagers": 20})
        #...
```

Les lycanthropes peuvent se trouver dans deux états: transformés ou non.
Afin de les différentier, changer la taille des lycanthropes transformés
et faites la passer à 6. Initialement, les lycanthropes ne sont pas
transformés. Ils se transforment de manière aléatoire, avec une
probabilité de 10%. Pour ajouter un nouveau comportement au lycanthrope,
il vous faudra modifier la méthode `step`.

Lorsqu'un lycanthrope est transformé, il peut s'attaquer à une autre
personne si celle-ci est à une portée de 40. Modifiez à nouveau la
méthode `step` de manière à permettre à un lycanthrope de
s'attaquer à une personne. Pour implémenter ce nouveau comportement, je
vous recommande vivement de vous appuyer sur des *list comprehensions*.
Si vous ne connaissez pas les list comprehensions, *merci de l'indiquer
dans le fichier* `responses.md`. L'autre difficulté de ce réflexe est
qu'il agit sur l'autre personne. Lorsqu'un lycanthrope attaque une
personne, il la transforme en lycanthrope.

**Question 1-** Comment avez-vous fait pour que l'autre agent soit
modifié? Cela vous paraît-il compatible de la définition d'agent que
vous avez vue en cours? Argumentez votre réponse.

*Insérez votre réponse ici*

Les apothicaires
----------------

Créez une classe `Cleric`. Ces agents ont les mêmes capacités
de déplacement que les villageois. Donnez-lui pour aspect un cercle de
rayon 3 et de couleur verte. Les apothicaires ont un comportement de
soin, leur permettant de rechanger un lycanthrope en humain à condition
que ce dernier soit à une distance de 30 et qu'elle ne soit pas
transformée. La simulation prend en compte un unique `Cleric`.

Les chasseurs
-------------

Créez une classe `Hunter`. Ces agents sont capables de se
déplacer de la même manière que les villageois. Dans la simulation, les
chasseurs seront représentés par un cercle noir de rayon 3. Implémentez
le comportement de chasse des chasseurs: ils sont capables de tuer un
lycanthrope si celui-ci est transformé et s'il se trouve à une distance
de 40. Tuer un agent se fait en le supprimant du
`Model.schedule`. Il y a 2 chasseurs dans notre simulation.

La simulation est maintenant complète! Vous pouvez la lancer et en
observer le résultat.

**Question 2-** Commentez le résultat de la simulation: Vers quoi le
système converge-t-il? En combien de cycles? À votre avis, quel est
l'impact de la présence de l'apothicaire? Celui de la quantité d'agents
de chaque espèce? Justifiez votre réponse

*Insérez votre réponse ici*

Expérimentations
================

Maintenant que vous avez exprimé des conjectures, il va falloir les
tester. Pour cela, il va falloir améliorer les simulations.

Graphiques
----------

Nous allons commencer par créer des indicateurs permettant de mesurer
plus finement la manière dont le système évolue. Mesa intègre un
environnement d'expérimentation que nous allons exploiter.

Nous allons laisser l'affichage de la simulation et afficher au-dessous
les graphiques. Un graphique est un `VizualizationElement`, et
plus particulièrement un `ChartModule`. En parallèle, il va
falloir rassembler des informations que nous souhaitons afficher. Pour
ce faire, nous allons utiliser un `DataCollector`. Le
DataCollector est un nouveau champ du modèle. Attention, gardez le nom
du champ que vous allez utiliser, il vous sera nécessaire par la suite.

L'utilisation du DataCollector se fait en l'initialisant dans le modèle.
Le constructeur de DataCollector prend en entrée un dictionnaire de
`model_reporters`, ayant d'un côté un nom (qu'il vous faudra
aussi garder en tête) et une fonction associant au modèle une valeur.
Notez que dans ce cadre, l'utilisation des `lambda` vous sera
très utile. Si vous ne connaissez pas les lambda expressions, *merci de
l'indiquer dans le fichier* `responses.md`.

Implémentez un graphique affichant le nombre de personnes (non
lycanthropes) en fonction du temps. Lancez à nouveau la simulation. Vous
voyez maintenant deux parties, incluant la courbe que vous avez créée.
Lorsque vous lancez la simulation, vous voyez la valeur de votre
métrique évoluer en direct, comme le montre la figure ci-dessous.

![Graphe](./chart.PNG)

Retournez à la fenêtre d'édition. Nous allons ajouter d'autres métriques
à notre page, afin de créer un tableau de bord qui permette de se faire
une meilleure idée de l'évolution du système. Tentez d'ajouter d'autres
graphes à l'onglet. Ajoutez trois graphes au tableau de bord
représentant le nombre de lycanthropes, le nombre de lycanthropes
transformés et le nombre total d'agents.

**Question 3-** Enregistrez les courbes encliquant avec le bouton droit
sur la courbe, puis "Enregistrez l'image sous..." et comparez ces
résultats à vos conjectures. Qu'en concluez-vous?

*Insérez votre réponse ici*

Variations de paramètres
------------------------

Voyons maintenant comment on peut faire varier les paramètres pour
observer leur influence sur l'expérimentation. Un moyen est de changer à
la main les valeurs entrées dans le code et relancer les simulations les
unes après les autres. Mais cela n'est pas pratique, et demande de
revenir sans arrêt au code. Il est donc possible d'ajouter des
paramètres à l'expérimentation afin de les faire varier directement dans
la perspective correspondante. Pour cela, il faut les déclarer
directement dans le lancement du serveur, dans les paramètres permettant
de lancer le serveur et changer le type des paramètres par un
`ModularVisualization.UserSettableParameter`. Ici, nous
utiliserons des `slider`, des sélecteurs utilisés pour les plages de
valeur:

```python
mesa.visualization.ModularVisualization.UserSettableParameter('slider',
        "name of slider", default_value, minimal_value, maximal_value, interval)
```

**Question 4-** Ajoutez aux paramètres le nombre de villageois sains, le
nombre de lycanthropes, le nombre de chasseurs et le nombre
d'apothicaires. Enregistrez les courbes qui vous paraissent pertinentes
et commentez-les. Cela correspond-il à vos hypothèses? Qu'en
concluez-vous?


*Insérez votre réponse ici*

**Question 5-** Sans faire les expériences associées, quels sont, selon
vous, les paramètres qui auraient une influence sur le résultat de la
simulation? Argumentez ces hypothèses.

*Insérez votre réponse ici*

Plan d'expérience
-----------------

Même en ajoutant les paramètres à la fenêtre de simulation, il peut
s'avérer long et fastidieux de tester une batterie de valeurs pour un
même paramètre. mesa intègre le moyen de créer plusieurs simulations en
faisant varier (ou non) ces paramètres. Pour cela, nous allons utiliser
les `batch`.

Dans cette partie, nous cherchons à évaluer l'impact du nombre
d'apothicaires dans un village avec 1 chasseur.

**Question 6-** Formulez une hypothèse argumentée sur le résultat de
cette expérience.

*Insérez votre réponse ici*

Pour cela, créez une nouvelle fonction `run_batch()` après la
fonction `run_single_server()`. Cette fonction créera d'abord
un dictionnaire qui donnera pour chaque paramètre de la fonction
`__init__()` du modèle `Village` une plage de valeur.
Les valeurs de `n_villagers`, `n_werewolves`,
`n_hunters`, respectivement à 50, 5 et 1. Le paramètre
`n_clerics` variera lui dans `range(0, 6, 1)`. Une
fois ce dictionnaire créé, instanciez un `Batchrunner`. Voici
la signature de son constructeur:
`BatchRunner(model, params_dict, model_reporter)`. Vous
utiliserez le même `model_reporter` que pour le modèle
individuel. Lancez le batchrun grâce à la méthode
`BatchRunner.run_all()`

Ce `model_reporter` vous permettra de récupérer une
`pandas.DataFrame` en utilisant la méthode
`BatchRunner.get_model_vars_dataframe()`.

**Question 7-** Comment interprétez-vous les résultats de cette
expérience? Qu'en concluez-vous?

*Insérez votre réponse ici*

Bonus
-----

Supposons que l'on souhaite faire varier toutes les variables, sur
toutes les valeurs permises par les `slider`; quel problème
voyez-vous? Comment peut-on le résoudre? Cherchez dans la documentation
de mesa et implémentez votre solution.

*Insérez votre réponse ici*

**Si certains éléments du framework mesa vous ont posé problème, merci
de l'indiquer à la fin du fichier** `reponses.md`
